# KorQuAD-Multitask-QuestionAnswer-Generation

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Environment setup
Setting up Google drive as working directory and installing packages.

In [2]:
%cd /content/drive/MyDrive/QnA_Multitask

/content/drive/MyDrive/QnA_Multitask


In [3]:
!pip install --quiet transformers==4.3.0
!pip install --quiet tokenizers==0.10.3
!pip install --quiet torchtext==0.6.0
!pip install --quiet pytorch-lightning==1.2.10
!pip install --quiet torchmetrics==0.6.0

     |████████████████████████████████| 1.8 MB 12.6 MB/s 
     |████████████████████████████████| 880 kB 55.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.0 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 1.3 MB 28.4 MB/s 
     |████████████████████████████████| 841 kB 11.2 MB/s 
     |████████████████████████████████| 176 kB 61.1 MB/s 
     |████████████████████████████████| 829 kB 57.7 MB/s 
     |████████████████████████████████| 329 kB 13.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 1.2.10 requires torchmetrics==0.2.0, but you have torchmetrics 0.6.0 which is incompatible.


In [4]:
# Import packages
from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np

import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )

In [5]:
pl.seed_everything(42)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


42

# Loading model for generating question

In [6]:
MODEL_NAME = 'paust/pko-t5-base'
SOURCE_MAX_TOKEN_LEN = 500
TARGET_MAX_TOKEN_LEN = 80
SEP_TOKEN = '<sep>'

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens(SEP_TOKEN)
TOKENIZER_LEN = len(tokenizer)

Downloading:   0%|          | 0.00/2.90M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/209 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [7]:
class QGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=LEARNING_RATE)

In [8]:
def generate(qgmodel: QGModel, answer: str, context: str) -> str:
    source_encoding = tokenizer(
        '{} {} {}'.format(answer, SEP_TOKEN, context),
        max_length=SOURCE_MAX_TOKEN_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    generated_ids = qgmodel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=1,
        max_length=TARGET_MAX_TOKEN_LEN,
        repetition_penalty=1.0,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)

In [9]:
checkpoint_path = 'checkpoints/best-checkpoint-v2.ckpt'

best_model = QGModel.load_from_checkpoint(checkpoint_path)
best_model.freeze()
best_model.eval()

print()

Downloading:   0%|          | 0.00/728 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

In [12]:
def show_result(generated: str):
    print('Generated: ', generated)

In [18]:
context = """제롬 파월 미국 연방준비제도(Fed·연준) 의장이 전 세계 투자자들의 이목이 집중된 잭슨홀 연설에서 공격적인 금리인상을 이어가겠다고 밝혔다. 경기 침체를 감수하고서라도 치솟는 물가를 잡는 데 주력하겠다는 뜻을 재확인한 것이다.

파월 의장의 매파(hawkish·통화긴축 선호)적인 발언에 연준이 경기 침체에 대응해 내년 하반기부터 금리인하에 돌입할 것이란 시장 기대는 사라졌다. 연준이 연말까지 큰 폭의 기준금리 인상을 단행하는 것은 물론, 내년까지도 높은 수준의 금리를 유지할 것으로 예상된다. 월가에서는 미국의 최종금리가 연 4%에 가까운 수준까지 높아질 것이라고 전망했다.

이미 초강세인 달러화의 독주는 더 심화될 것이란 전망이 나온다. 최근 1330~1340원대로 뛴 원·달러 환율은 연말까지 높은 수준을 지속할 가능성이 높아졌다. 미국의 고금리 정책으로 글로벌 경기 침체 우려가 되살아나면서 고(高)환율이 지속되고, 한·미 금리역전 현상이 두드러질 경우 올 하반기 우리나라 수출과 물가, 경상수지 등 경제 전반에도 부정적인 영향을 미칠 것으로 보인다.

"""
input_answer = '[MASK]'
        
generated = generate(best_model, input_answer, context)
        
show_result(generated)

Generated:  잭슨홀 연설 <sep> 미국 연방준비제도 의장이 공격적인 금리인상을 이어가겠다고 밝힌 연설은?
